# NSU, vaje 2: meta učenje

### A: Spletna stran OpenML in knjižnica **openml**, priprava podatkov

A.1 Dobi podatke z OpenML-ja (https://www.openml.org/).
Najdi vsa podatkovja, ki imajo med 100 in 200 primerov in med 4 do 100 znacilk. Pomagaj si s funkcijo **openml.datasets.list_datasets**. Podaj ji argumente
- number_instances (npr. number_instances="\<x>..\<y>", kjer sta x in y najmanjse in najvecje dovoljeno stevilo primerov, npr. "10..20")
- number_features (analogno number_instances)
- output_format="dataframe"
in si oglej dobljeni pandasov Dataframe. Eden od stolpcev v njem je did (data ID).

Za opis OpenML funkcij lahko uporabis dokumentacijo na https://docs.openml.org/Python-API/

In [1]:
from openml.datasets import list_datasets, get_datasets
import pandas as pd
df = list_datasets(number_instances="100..200", number_features="4..100", output_format="dataframe")
df

,did,name,version,uploader,status,format,MajorityClassSize,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures
10,10,lymph,1,1,active,ARFF,81.0,8.0,2.0,4.0,19.0,148.0,0.0,0.0,3.0,16.0
48,48,tae,1,1,active,ARFF,52.0,3.0,49.0,3.0,6.0,151.0,0.0,0.0,3.0,3.0
55,55,hepatitis,1,1,active,ARFF,123.0,2.0,32.0,2.0,20.0,155.0,75.0,167.0,6.0,14.0
61,61,iris,1,1,active,ARFF,50.0,3.0,50.0,3.0,5.0,150.0,0.0,0.0,4.0,1.0
62,62,zoo,1,1,active,ARFF,41.0,7.0,4.0,7.0,17.0,101.0,0.0,0.0,1.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43839,43839,IRIS-flower-dataset,1,30125,active,arff,NaN,NaN,NaN,NaN,5.0,150.0,0.0,0.0,4.0,0.0
43859,43859,iriiiiiis,2,28351,active,ARFF,50.0,NaN,50.0,3.0,5.0,150.0,0.0,0.0,4.0,1.0
44151,44151,Iris,51,30495,active,arff,50.0,NaN,49.0,3.0,5.0,149.0,0.0,0.0,4.0,0.0
44154,44154,iris_reproduced,1,30495,active,arff,50.0,NaN,50.0,3.0,5.0,150.0,0.0,0.0,4.0,1.0


A.2 Nalozi ta podatkovja s funkcijo **openml.datasets.get_datasets**, tako da uporabis ID-je podatkovij iz prejsnje tocke. To bo prvic morda trajalo nekaj minut. Ker se bodo nalozena podatkovja shranila tudi na disk racunalnika, bo vsak naslednji klic precej hitrejsi.

In [2]:
datasets = get_datasets(df['did'])

A.3 Iz zbirke podatkovij odstrani tista, ki niso primerna za klasifikacijo. Pomagas si lahko s klicem **podatkovje.get_data()**, ki vrne

- x: pandasov Dataframe znacilk, ki vkljucuje tudi ciljno spremenljivko, ce argument target in podan
- y: stolpec, ki podaja vrednosti ciljne spremenljivke, ce je argument target podan, in None sicer
- nominalni: seznam vrednosti True/False, ki pove, ali je i-ti atribut nominalen
- atributi: seznam imen atributov

Ciljno spremenljivko posameznega podatkovja najdemo z **podatkovje.default_target_attribute**.

Iz vseh podatkovij iz prejsnje naloge odstrani tista, ki imajo
- neznano (None) ali numericno ciljno spremenljivko,
- več ciljnih spremenljivk.

Pazi, da obdržiš le eno razlicico podatkov: "iris" se npr. pojavi vec kot 40-krat. Ime podatkovja je shranjeno v
polju "name" (do njega torej dostopamo s "podatkovje.name")
Ker scikit ne podpira nominalnih znacilk, odstrani tudi vsa podatkovja, ki vsebujejo nominalne znacilke.


In [3]:
new_datasets = []
names = set()
from pandas.api.types import is_numeric_dtype
i = 0
for dataset in datasets:
    if dataset.name in names:
        continue
    else:
        names.add(dataset.name)

    y = dataset.default_target_attribute

    if y is not None:
        if ',' in y:
            continue
    
        x,tg,z,w = dataset.get_data()

        if len([1 for i in z if i]) != 1:
            continue
        x,tg,z,w = dataset.get_data(target=y)

        if not is_numeric_dtype(tg):
            new_datasets.append(dataset)
    



In [4]:
len(new_datasets)

27

### B - Priprava ciljnih spremenljivk za meta učenje 
Naša ciljna spremenljivka bo uspešnost posameznih metod strojnega učenja na različnih podatkih. 

Na vseh OK podatkovjih pozeni modele kNN (**sklearn.neighbors.KNeighborsClassifier**), odlocitveno drevo (**sklearn.tree.DecisionTreeClassifier**) in naivnega Bayesa (**sklearn.naive_bayes.GaussianNB**).
Za smiselno oceno kvalitete je treba razbiti podakovje na ucno in testno mnozico. V testno mnozico daj 1/4 primerov. 
Kvaliteto napovedi izmeri s tocnostjo.
Ce se bo pri ucenju/napovedovanju koda sesula, to morda pomeni, da s podatki ni vse v redu in bi jih bilo potrebno prej
odfiltrirati.
Dobljene rezultate shrani v tabelo s stolpci

**ime podatkovja,knn,drevo,bayes,najboljsi**

kjer stolpci 2-4 podajajo tocnosti modelov na tesni podmnozici podatkovja, stolpec najboljsi pa ime najboljsega
modela. Pandasov dataframe lahko preprosto shraniš v CSV datoteko s funkcijo **df.to_csv**.

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
import numpy as np

In [6]:
def f(method, d1, d2, d3, d4):
    a = method().fit(d1, d3)
    return a.score(d2, d4)

allscores = []
methods = ['KNeighborsClassifier', 'DecisionTreeClassifier', 'GaussianNB']

for dataset in new_datasets:

    x,y,nominal,w=dataset.get_data(target=dataset.default_target_attribute)
    r = train_test_split(x, y, train_size=0.75)

    scores = [dataset.name, f(KNeighborsClassifier, *r), f(DecisionTreeClassifier, *r), f(GaussianNB, *r)]
    scores.append(np.argmax(scores[1:]))
    allscores.append(scores)





In [7]:
allscores

[['iris', 0.9473684210526315, 0.9210526315789473, 1.0, 2],
 ['zoo', 0.8846153846153846, 0.9615384615384616, 0.9615384615384616, 1],
 ['wine', 0.8666666666666667, 0.9333333333333333, 1.0, 2],
 ['hayes-roth', 0.6, 0.75, 0.6, 1],
 ['transplant', 1.0, 1.0, 1.0, 0],
 ['kc1-top5', 0.8648648648648649, 0.8378378378378378, 0.8378378378378378, 0],
 ['kc1-binary', 0.7027027027027027, 0.7567567567567568, 0.6216216216216216, 1],
 ['datatrieve', 0.8787878787878788, 0.8787878787878788, 0.7878787878787878, 0],
 ['MyIris', 0.9736842105263158, 0.8947368421052632, 0.9473684210526315, 0],
 ['KungChi3', 0.9032258064516129, 0.8709677419354839, 0.9032258064516129, 0],
 ['KnuggetChase3',
  0.7959183673469388,
  0.6530612244897959,
  0.8367346938775511,
  2],
 ['MindCave2', 0.6875, 0.65625, 0.71875, 2],
 ['breast-tissue',
  0.4074074074074074,
  0.3333333333333333,
  0.4074074074074074,
  0],
 ['fertility', 0.84, 0.8, 0.88, 2],
 ['parkinsons', 0.8979591836734694, 0.8571428571428571, 0.6530612244897959, 0],
 ['

### C Priprava napovednih spremenljivk za meta učenje

Pri pripravi meta znacilk nam bo pomagal paket pymfe https://pypi.org/project/pymfe/.
Najbolj koristno bo orodje **MFE**, ki mu v argumentu **groups** povemo, katere tipe meta značilk želimo.
Po nastavitvi moramo poklicati še njegovi funkciji **fit** in **extract**.

Najprej preizkusimo pymfe na enem izmed podatkovij v naši zbirki.

C.1 Naredi meta značilke iz podatkov (tip značilk "general" in "info-theory"). Oglej si ustvarjene značilke. 
Preizkusiš lahko tudi druge možnosti, npr. "statistical".

Opozorilo, da nekaterih značilk ni mogoče izračunati, ni nič nenavadnega.

In [8]:
from pymfe import mfe
x,y,z,w = new_datasets[0].get_data(target=new_datasets[0].default_target_attribute)

a = mfe.MFE(groups=('general', 'info-theory'))
a.fit(x.values.tolist(),y.values.tolist())
names = a.extract()


C.2 Naredi meta značilke na podlagi enostavnega modela - odločitvenega drevesa, naučenega na celotni množici. Model ustvarimo in naučimo samo, značilke pa iz naučenega drevesa pridobimo z **MFE** in nastavitvijo "model-based" ter funkcjo **extract_from_model**.

In [9]:
a = mfe.MFE(groups='model-based')
mod = DecisionTreeClassifier()
mod.fit(x,y)
names2 = a.extract_from_model(mod)

C.3 Izračunaj vse tri vrste značilk za vsa podatkovja v naši zbirki in jih shrani v tabelo. Naredi tabelo s stolpci

**name,znacilka1,znacilka2,...,znacilkaN**, 

kjer je N stevilo znacilk (imena stolpcev 2-N niso znacilkaI, ampak dejanska imena znacilk, ki se ustvarijo.)
Tabela lahko vsebuje manjkajoce vrednosti. Tabelo shrani tudi v datoteko.

In [10]:
arr = [['name']+names[0]+names2[0]]
for dataset in new_datasets:
    m = mfe.MFE(groups=('general','info-theory'))
    result = [dataset.name]
    x,y,z,w = dataset.get_data(target=dataset.default_target_attribute)
    result.extend(m.fit(x.values.tolist(),y.values.tolist()).extract()[1])
    m = mfe.MFE(groups='model-based')
    mod = DecisionTreeClassifier()
    mod.fit(x,y)
    result.extend(a.extract_from_model(mod)[1])
    
    arr.append(result)


/home/tadej/.local/lib/python3.10/site-packages/pymfe/_internal.py:1568: UserWarning: It is not possible make equal discretization
  warnings.warn("It is not possible make equal discretization")
/home/tadej/.local/lib/python3.10/site-packages/pymfe/_internal.py:1568: UserWarning: It is not possible make equal discretization
  warnings.warn("It is not possible make equal discretization")
/home/tadej/.local/lib/python3.10/site-packages/pymfe/_internal.py:731: RuntimeWarning: Can't summarize feature 'nodes_per_level' with summary 'sd'. Will set it as 'np.nan'.
  warnings.warn(
/home/tadej/.local/lib/python3.10/site-packages/pymfe/_internal.py:731: RuntimeWarning: Can't summarize feature 'nodes_repeated' with summary 'sd'. Will set it as 'np.nan'.
  warnings.warn(
/home/tadej/.local/lib/python3.10/site-packages/pymfe/_internal.py:731: RuntimeWarning: Can't summarize feature 'tree_imbalance' with summary 'sd'. Will set it as 'np.nan'.
  warnings.warn(
/home/tadej/.local/lib/python3.10/site-

In [11]:
pd.DataFrame(arr).to_csv('vaje-2-cenilke.csv', header=False, index=False)